In [1]:
# IMPORTS
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, LeakyReLU

In [6]:
# Load information about the stock
train = pd.read_csv('E:\Coding\MachineLearning\CSVFile\VNI.csv')
test = pd.read_csv('E:\Coding\MachineLearning\CSVFile\VNItest.csv')

In [7]:
train["Change %"] = train["Change %"].str.replace('%', '').astype(float)
train["Price"] = train["Price"].str.replace(',', '').astype(float)
train["Open"] = train["Open"].str.replace(',', '').astype(float)
train["High"] = train["High"].str.replace(',', '').astype(float)
train["Low"] = train["Low"].str.replace(',', '').astype(float)
train["Vol."] = train["Vol."].str.replace('K', '0')
train["Vol."] = train["Vol."].str.replace('M', '0000')
train["Vol."] = train["Vol."].str.replace('.', '').astype(float)

C:\Users\PC\AppData\Local\Temp\ipykernel_1516\1822012783.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train["Vol."] = train["Vol."].str.replace('.', '').astype(float)


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      130 non-null    object 
 1   Price     130 non-null    float64
 2   Open      130 non-null    float64
 3   High      130 non-null    float64
 4   Low       130 non-null    float64
 5   Vol.      130 non-null    float64
 6   Change %  130 non-null    float64
dtypes: float64(6), object(1)
memory usage: 7.2+ KB


In [15]:
trainX = pd.to_datetime(train["Date"])
trainY = train["Price"]
testX = test["Date"]
testY = test["Price"]

In [19]:
# Parameters
nFeatures = trainX.shape
epochs = 20
batchSize = 1000
nOutput = 1
kernelSize = 1

In [20]:
# Model Building
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=kernelSize, padding='same', activation='relu', input_shape=(nFeatures, 1)))
model.add(Conv1D(filters=64, kernel_size=kernelSize, padding='same'))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling1D(pool_size=(1)))
model.add(Conv1D(filters=128, kernel_size=kernelSize, padding='same'))
model.add(LeakyReLU(alpha=0.01))
model.add(Flatten())
model.add(Dense(256,))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.8))
model.add(Dense(nOutput, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Fit the model
fitReturn = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=epochs, batch_size=batchSize, verbose=1)

TypeError: Dimension value must be integer or None or have an __index__ method, got value '(130,)' with type '<class 'tuple'>'

In [ ]:
plt.title('Loss')
plt.grid(True)
plt.xlabel('Epochs')
plt.ylabel('Values')
plt.plot(fitReturn.history['loss'], 'blue', label='Train Loss')
plt.plot(fitReturn.history['val_loss'], 'red', label='Test Loss')
plt.legend()
plt.show()
plt.close()

In [ ]:
# Predictions
predictY = model.predict_classes(testX, verbose=0)
predictY = predictY[:, 0]

# Basic Counting
testY0 = (testY == 0).sum()
testY1 = (testY == 1).sum()
print("Test Set - Sell signal : "+str(testY0))
print("Test Set - Buy signal  : "+str(testY1))
print("="*40)
predictY0 = (predictY == 0).sum()
predictY1 = (predictY == 1).sum()
print("Predicted - Sell signal : "+str(predictY0))
print("Predicted - Buy signal  : "+str(predictY1))

In [ ]:
# Computing Scores
accuracy = accuracy_score(testY, predictY)
precision = precision_score(testY, predictY)
recall = recall_score(testY, predictY)
f1 = f1_score(testY, predictY)
matrix = confusion_matrix(testY, predictY)
print('Accuracy: '+str(accuracy))
print('Precision: '+str(precision))
print('Recall: '+str(recall))
print('F1 Score: '+str(f1))
print(matrix)